# Best areas to go out in a city at night
With the code in this notebook you can find areas with a high density of nightlife venues in a city you are interested in.
Perfect for pub crawls and nightclub hopping!
Data is optained from Foursquare.

In [1]:
# importing necessary packages
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
from pylab import rcParams
from mpl_toolkits.basemap import Basemap
%matplotlib inline

# import k-means from clustering stage
#from sklearn.cluster import KMeans

import folium # map rendering library

import geopy
import geocoder 
from geopy.geocoders import Nominatim

from sklearn.cluster import DBSCAN 
from sklearn.datasets.samples_generator import make_blobs 
from sklearn.preprocessing import StandardScaler 
import sklearn.utils

print('Libraries imported.')

Libraries imported.


### Get a location

In [2]:
# let the user decide which city or region he/she wants to explore
print('Please enter a city or address!')
address = input()
print('Perfect, you entered ' + address) 

Please enter a city or address!
Miami Beach
Perfect, you entered Miami Beach


In [3]:
# get geolocation of city 
df_city = pd.DataFrame(columns = ['City','Latitude','Longitude'])

geolocator = Nominatim(user_agent="germany_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address , latitude, longitude))

The geograpical coordinates of Miami Beach are 25.7929198, -80.1353006.


### Get nightlife venues nearby with Foursquare API

In [4]:
# Foursquare API credentials
CLIENT_ID = 'Bxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'Wxxxxxxxxxxx' # your Foursquare Secret
ACCESS_TOKEN = '2xxxxxxxxxxx' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version

In [5]:
# choose categories by category ID
category = {'4bf58dd8d48988d11f941735', # Nightclub
            '52e81612bcbc57f1066b7a06', # Irish Pub
            '4bf58dd8d48988d1d5941735', # Hotel Bar
            '4bf58dd8d48988d121941735', # Lounge
            '4bf58dd8d48988d11e941735', # Cocktail Bar
            '4bf58dd8d48988d116941735', # Bar
            '4bf58dd8d48988d11b941735'} # Pub

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# creat the url to search for venues to go out to at night
df_night = pd.DataFrame()
for id in category:
    LIMIT = 1000
    radius = 10000
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        latitude, 
        longitude,
        ACCESS_TOKEN, 
        VERSION, 
        id, 
        radius, 
        LIMIT)
    #print(url)

    

    #get results of Foursquare request
    results = requests.get(url).json()
    
    venues = results['response']['venues'] # assign relevant part of JSON to venues
    dataframe = json_normalize(venues) # tranform venues into a dataframe

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1) # filter the category for each row
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns] # clean column names by keeping only last term
    
    df_night = df_night.append(dataframe_filtered, ignore_index=True)

df_night

C:\Users\jenni\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,address,categories,cc,city,country,crossStreet,distance,formattedAddress,id,labeledLatLngs,lat,lng,name,neighborhood,postalCode,state
0,1901 Convention Center Dr,Lounge,US,Miami Beach,United States,btwn Dade Blvd & 17th St,225,[1901 Convention Center Dr (btwn Dade Blvd & 1...,5a281c0a6e46503483635f57,"[{'label': 'display', 'lat': 25.79487228393554...",25.794872,-80.134697,UBS VIP Lounge,NaN,33139,FL
1,320 Lincoln Rd,Lounge,US,Miami Beach,United States,Washington Ave,489,"[320 Lincoln Rd (Washington Ave), Miami Beach,...",59eb13158fb09e268cf3ec3f,"[{'label': 'display', 'lat': 25.79052606938197...",25.790526,-80.131200,Mr. Jones Lounge,NaN,33139,FL
2,"1701 Collins Ave, Miami Beach",Hotel,US,Miami Beach,United States,at 17th St,657,"[1701 Collins Ave, Miami Beach (at 17th St), M...",4fcd4704e4b00c762cbcb75e,"[{'label': 'display', 'lat': 25.7925365, 'lng'...",25.792537,-80.128759,SLS South Beach,NaN,33139,FL
3,1131 Collins Ave,Mediterranean Restaurant,US,Miami Beach,United States,NaN,1245,"[1131 Collins Ave, Miami Beach, FL 33139, Unit...",5b8c8ae1d552c7002c4fa0b3,"[{'label': 'display', 'lat': 25.78234780231423...",25.782348,-80.131239,Rakija Lounge,NaN,33139,FL
4,1850 Collins Ave,Bar,US,Miami Beach,United States,NaN,589,"[1850 Collins Ave, Miami Beach, FL 33139, Unit...",5a3199023abcaf1afcd3a2a9,"[{'label': 'display', 'lat': 25.79475458119335...",25.794755,-80.129780,Moreno's Cuba,NaN,33139,FL
5,NaN,Lounge,US,Miami Beach,United States,NaN,739,"[Miami Beach, FL 33139, United States]",53c046db498e1923339c8402,"[{'label': 'display', 'lat': 25.78693068897602...",25.786931,-80.132119,Espanola Cigar Bar,NaN,33139,FL
6,NaN,Lounge,US,Miami Beach,United States,NaN,1310,"[Miami Beach, FL, United States]",59277510aa6c952299884a92,"[{'label': 'display', 'lat': 25.78163952956486...",25.781640,-80.131555,The Porch Bar & Bites,NaN,NaN,FL
7,3500 Collins Ave,Lounge,US,Miami Beach,United States,NaN,2148,"[3500 Collins Ave, Miami Beach, FL 33140, Unit...",5e45d273072dbc0008353cb0,"[{'label': 'entrance', 'lat': 25.80923, 'lng':...",25.809261,-80.123894,Gitano,NaN,33140,FL
8,111 NE 20th St,Lounge,US,Miami,United States,NaN,5744,"[111 NE 20th St, Miami, FL 33137, United States]",5fb920b4aed7f345b1e669cf,"[{'label': 'display', 'lat': 25.795828, 'lng':...",25.795828,-80.192525,Boho House,NaN,33137,FL
9,4441 Collins Ave,Lounge,US,Miami Beach,United States,NaN,3046,"[4441 Collins Ave, Miami Beach, FL 33140, Unit...",4e5094eed4c0b6da5d8f8eff,"[{'label': 'display', 'lat': 25.81757440263602...",25.817574,-80.122107,ÄrKadïã,NaN,33140,FL


### Density based clustering of venues

In [7]:
sklearn.utils.check_random_state(1000)
Clus_dataSet = df_night[['lat','lng']]
Clus_dataSet = np.nan_to_num(Clus_dataSet)
Clus_dataSet = StandardScaler().fit_transform(Clus_dataSet)

# Compute DBSCAN
db = DBSCAN(eps=0.17, min_samples=3).fit(Clus_dataSet)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
df_night["Clus_Db"]=labels

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels)) 


# A sample of clusters
df_night[["name","categories","lat","lng", "Clus_Db"]].head(5)

,name,categories,lat,lng,Clus_Db
0,UBS VIP Lounge,Lounge,25.794872,-80.134697,0
1,Mr. Jones Lounge,Lounge,25.790526,-80.131200,0
2,SLS South Beach,Hotel,25.792537,-80.128759,0
3,Rakija Lounge,Mediterranean Restaurant,25.782348,-80.131239,0
4,Moreno's Cuba,Bar,25.794755,-80.129780,0


### Visualization

In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the city center (or address)

col = ['red','blue','green','purple','orange','darkred','lightred','darkblue','darkpurple','lightgreen','darkgreen','cadetblue','lightblue','#ff7f50','#81d8d0','#ccff00','#7fe5f0','#ff0000','#ff80ed','#407294','#cbcba9','#468499','#f6546a','#afeeee','#008000','#660066','#cbbeb5','#ff4040','#bada55','#f5f5dc','beige']
# add the venues as blue circle markers
for lat, lng, category, name, cluster in zip(df_night.lat, df_night.lng, df_night.categories, df_night.name, df_night.Clus_Db):
    label = folium.Popup(str(name) + ' -\n' + str(category), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=col[cluster],
        popup= label,
        fill = True,
        fill_color=col[cluster],
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add a red circle marker to represent the center
folium.Marker(
    [latitude, longitude],
    popup='Center',
).add_to(venues_map)

# display map
venues_map

The Marker shows the location you typed in. The dots present venues where you can go out at night (bar, nightclub, pub,...). The colored clusters show areas with a high density of these venues and thus, suggest areas where you can visit many clubs, bars, pubs etc. Without walking to far. Perfect for a pub crawl or nightclub hopping! Venues that do not belong to a cluster are however shown as less visible beige dots.
Note, that there is a limit of 50 venues per request with the free account. If there wasn't a limit of 50 venues the clusters would be more informative.